In [ ]:
%reset
import numpy as np
import scipy.io as sio
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib inline
%pwd

In [ ]:
#SPECIFY THESE:
XN = 640 #NUMBER OF POINTS IN X 640
YN = 4  #NUMBER OF POINTS IN Y
ZN = 120 #NUMBER OF POINTS IN Z 120 #default 120

xmat = nc.Dataset('xmat.nc')['xmat'][:]
Qmat = nc.Dataset('Qmat.nc')['Qmat'][:] #Heat flux, negative out of the ocean
SFmat = nc.Dataset('SFmat.nc')['SFmat'][:] #Evaporative flux, negative out of the ocean

interpx1=xmat
interpx2=np.linspace(0,75000,XN) 

Qinterp=np.expand_dims(np.interp(interpx2,interpx1,Qmat),axis=0)
Qinterp=np.expand_dims(Qinterp,axis=0)
Q_input=np.repeat(Qinterp,YN,axis=1)

SFinterp=np.expand_dims(np.interp(interpx2,interpx1,SFmat),axis=0)
SFinterp=np.expand_dims(SFinterp,axis=0)
SF_input=np.repeat(SFinterp,YN,axis=1)

SST_input=(SF_input*0.0)+0.0059
SSS_input=(SF_input*0.0)+34.4615

In [ ]:
print(Q_input.shape)
print(SF_input.shape)
print(SST_input.shape)

In [ ]:
print(SF_input)

In [ ]:
fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.plot(interpx2/1000,Q_input[0,0,:]) 
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Value',fontsize=14)
plt.title('Heat flux [W/m^2]', fontsize=14)
plt.show()

fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.plot(interpx2/1000,SF_input[0,0,:]) 
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Value',fontsize=14)
plt.title('Evaporative flux [kg/(m^2s)]', fontsize=14)
plt.show()


fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.plot(interpx2/1000,SST_input[0,0,:]) 
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Value',fontsize=14)
plt.title('SST reference', fontsize=14)
plt.show()


fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.plot(interpx2/1000,SSS_input[0,0,:]) 
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Value',fontsize=14)
plt.title('SSS reference', fontsize=14)
plt.show()

In [ ]:
writing = nc.Dataset("forcing.nc","w",format="NETCDF3_64BIT_OFFSET")
TIME = writing.createDimension("TIME",None)
LAT = writing.createDimension("LAT",YN)
LON = writing.createDimension("LON",XN)

TIME = writing.createVariable("TIME","f8",("TIME"))
TIME[:] = 0.0
TIME.standard_name="time"
TIME.units="days since 0001-01-01 00:00:00"

evap = writing.createVariable("evap","f8",("TIME","LAT","LON"))
evap[:,:,:]=SF_input
evap.standard_name = "Salt flux forcing = evaporation (both negative)"
evap.units = "kg/(m^2*s)"

LW = writing.createVariable("LW","f8",("TIME","LAT","LON"))
LW[:,:,:]=0.0*SF_input
LW.standard_name = "Longwave flux downward"
LW.units = "W/m^2"

SW = writing.createVariable("SW","f8",("TIME","LAT","LON"))
SW[:,:,:]=0.0*SF_input
SW.standard_name = "Shortwave flux downward"
SW.units = "W/m^2"

sensible = writing.createVariable("sensible","f8",("TIME","LAT","LON"))
sensible[:,:,:]=Q_input
sensible.standard_name = "Sensible heat flux into ocean"
sensible.units = "W/m^2"

latent = writing.createVariable("latent","f8",("TIME","LAT","LON"))
latent[:,:,:]=0.0*SF_input
latent.standard_name = "Latent heat flux into ocean"
latent.units = "W/m^2"

liq_precip = writing.createVariable("liq_precip","f8",("TIME","LAT","LON"))
liq_precip[:,:,:]=0.0*SF_input
liq_precip.standard_name= "Liquid precipitation into ocean"
liq_precip.units = "kg/(m^2*s)"

froz_precip = writing.createVariable("froz_precip","f8",("TIME","LAT","LON"))
froz_precip[:,:,:]=0.0*SF_input
froz_precip.standard_name= "Frozen precipitation into ocean"
froz_precip.units = "kg/(m^2*s)"

liq_runoff = writing.createVariable("liq_runoff","f8",("TIME","LAT","LON"))
liq_runoff[:,:,:]=0.0*SF_input
liq_runoff.standard_name= "Liquid runoff into ocean"
liq_runoff.units = "kg/(m^2*s)"

froz_runoff = writing.createVariable("froz_runoff","f8",("TIME","LAT","LON"))
froz_runoff[:,:,:]=0.0*SF_input
froz_runoff.standard_name= "Frozen runoff into ocean"
froz_runoff.units = "kg/(m^2*s)"

SST = writing.createVariable("SST","f8",("TIME","LAT","LON"))
SST[:,:,:]=SST_input
SST.standard_name = "Reference surface SST"
SST.units = "Celsius"

SSS = writing.createVariable("SSS","f8",("TIME","LAT","LON"))
SSS[:,:,:]=SSS_input
SSS.standard_name = "Reference surface SSS"
SSS.units = "psu"


writing.close()

In [ ]:
#NEW FILE CHECK
filenew = nc.Dataset('forcing.nc')
print(filenew)
SF_new=filenew.variables['evap'][:]
print(SF_new.shape)
Q_new=filenew.variables['sensible'][:]
SST_new=filenew.variables['SST'][:]
print(Q_new.shape)

In [ ]:
print(SST_new)